In [ ]:
import matplotlib.pyplot as plt
import itertools
import json
import argparse
import numpy as np
import seaborn as sns

from plot_utils import extract_measures, extract_utterance, get_distributions, get_x_positions, order_lists, plot_reward, plot_training_curve 

sns.set()
# plt.set
# plt.figure.

%matplotlib inline

dir_ = 'final/new/'

filenames = {
    'self_both': dir_ + 'log_20191116_215417final_self_2.log',
    'self_linguistic': dir_ +'log_20191105_181425final_self_disable-proposal.log',
    'self_proposal': dir_ + 'log_20191106_174032final_self_disable-comms.log',
    'self_none': dir_ + 'log_20191109_114924final_self_disable-proposal_disable-comms.log',
    
    'prosoc_both': dir_ + 'log_20191103_201228final_prosoc.log',
    'prosoc_linguistic': dir_ + 'log_20191104_193106final_prosoc_disable-proposal.log',
    'prosoc_proposal': dir_ + 'log_20191104_145248final_prosoc_disable-comms.log',
    'prosoc_none': dir_ + 'log_20191107_200112final_prosoc_disable-proposal_disable-comms.log'  
}

filenames_memory_comp = {
    'prosoc_none': dir_ + 'log_20191108_182251final_prosoc_disable-proposal_disable-comms_memory-comp.log',
    'prosoc_linguistic': dir_ + 'log_20191111_090756final_prosocial_memory-comp_disable-proposal.log',
    'prosoc_both': dir_ + 'log_20191110_105512final_prosocial_memory-comp.log',
    'prosoc_proposal': dir_ + 'log_20191112_174358final_prosocial_memory-comp_disable-comms.log',
    
    'self_proposal':dir_ + 'log_20191113_185154final_self_memory-comp_disable-comms.log',
    'self_linguistic': dir_ + 'log_20191114_095448final_self_memory-comp_disable-proposal.log',
    'self_both': dir_ + 'log_20191115_090601final_self_memory-comp.log',
    'self_none': dir_ + 'log_20191116_113959final_self_disable-proposal_disable-comms_memory-comp.log'
}

output_dir = 'figs/'
    
def double_check(filenames):
    for key, filename in filenames.items():
        try:
            with open(filename, 'r') as f:
                    for n, line in enumerate(f):
                        if n == 0:
                            msg = line
                        break
        except Exception as e:
            print(e)
            msg = 'Somethings wrong in here'
        print(key, msg)

def gen_iter(exclude_sociality=None, exclude_channels=None):
    """
    exclude_sociality: str
    exclude_channels: list of str
    """
    socialities = ['self', 'prosoc']
    channels = ['proposal', 'linguistic', 'both', 'none']
    if exclude_sociality:
        socialities.remove(exclude_sociality)
    if exclude_channels:
        for channel in exclude_channels:
            channels.remove(channel)
    return itertools.product(socialities, channels)

print('double check filenames:')
double_check(filenames)
print('\ndouble check filenames_memory_comp:')
double_check(filenames_memory_comp)

In [ ]:
def plot_reward_all(filenames, prefix=''):
    for sociality, channel in gen_iter():
        key = sociality + '_' + channel
        print(key)
        filename = filenames[key] if key in filenames else None

        if filename:
            output = '{}reward_{}_{}.png'.format(output_dir, key, prefix)
            plot_reward(filename, 0, 1, key, 200000, labels=None, output_file=output)   

In [ ]:
plot_reward_all(filenames)
plot_reward_all(filenames_memory_comp, 'memory-comp')

In [ ]:
def joint_reward_success(filenames):

    """
    FOR TABLE 2
    
    keys:
    self_proposal, self_linguistic, self_both, self_none,
    prosoc_proposal, prosoc_linguistic, prosoc_both, prosoc_none
    
    values:
    filenames
    
    Joint reward success and average number of turns taken for paired agents negotiating
    with random game termination, varying the agent reward scheme and communication channel.
    """
    
    from_paper = {'self_proposal': 0.87, 'self_linguistic': 0.75, 'self_both': 0.87, 'self_none': 0.77,
                  'prosoc_proposal': 0.93,  'prosoc_linguistic': 0.99, 'prosoc_both': 0.92, 'prosoc_none': 0.95}
    
    data = {}
    
    for sociality, channel in gen_iter():
        key = sociality + '_' + channel
        filename = filenames.get(key, None)

        if filename:
            extracted = extract_measures(filenames[key], ['test_reward'])
            joint_reward = np.mean(extracted['test_reward'])

        else:
            joint_reward = -1
        data[key] = {'joint_reward': joint_reward}
            
    for sociality, channel in gen_iter():
        key = sociality + '_' + channel
        print(sociality + ' ' + channel)
        print('\tour:       {}'.format(data[key]['joint_reward']))
        print('\tfrom paper: {}'.format(from_paper[key]))
    return data
    
    

In [ ]:
print('without memory-comp:\n')
joint_reward_success(filenames)
print('\n\nmemory-comp:')
joint_reward_success(filenames_memory_comp)

In [ ]:
def plot_training_curve_all(filenames, prefix=''):
    """
    FOR FIGURE 2a
    
    Training curves for SELF-INTERESTED agents learning to negotiate under the various com- munication channels.
    """

    for sociality, channel in gen_iter():
        key = sociality + '_' + channel
        filename = filenames.get(key, None)
        if filename:
            output = '{}training_curve_{}{}.png'.format(output_dir, key, prefix)
            plot_training_curve(filename, min_y=0, max_y=1, title='', max_x=200000, labels=None, output=output)
        

In [ ]:
plot_training_curve_all(filenames_memory_comp, '_memory-comp')
plot_training_curve_all(filenames)

In [ ]:
def plot_utterance(distribution, turn, vocab_len=10, utter_len=6, output_dir=False, key=False):
    labels = [str(i) for i in range(utter_len)]
    
    x = np.arange(utter_len)
    width = 4
    positions = get_x_positions(vocab_len, utter_len, width=width, outer_width=2).T
    
    plt.clf()
    fig, ax = plt.subplots()
    
    plt.ylim((0, 1.05))
    reacts = []
    
    for i in range(vocab_len):
        values = distribution[turn, :, i]
        l = positions[i]
        reacts.append(ax.bar(l, values, width, label=str(i)))
    ax.legend()
    if output_dir and key:
        output = '{}unigrams{}.png'.format(output_dir, key)
        plt.savefig(output)
        
    def autolabel(rects):
        """
        Attach a text label above each bar in *rects*, displaying its height.
        from: https://matplotlib.org/3.1.1/gallery/lines_bars_and_markers/barchart.html#sphx-glr-gallery-lines-bars-and-markers-barchart-py
        """
        for rect in rects:
            height = rect.get_height()
            ax.annotate('{}'.format(height),
                        xy=(rect.get_x() + rect.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom')
            
    ax.set_ylabel('Occurance')
    ax.set_title('Symbols distribution in position')
#     ax.set_xticks(np.arange(labels))
    ax.set_xticklabels(labels)
#     fig.tight_layout()


def plot_unigram_statistics(filenames, output_dir=False, memory_comp=False):
    """
    FOR FIGURE 3a
    
    Unigram statistics of symbol usage broken down by turn and by position within the utterance
    for prosocial agents communicating via the linguistic channel.
    """
    memory_comp = ['', 'memory_comp'][memory_comp]

    for sociality, channel in gen_iter(exclude_channels=['proposal', 'none']):
        key = sociality + '_' + channel
        filename = filenames.get(key, None)
        if filename:
            print(filename)
            extracted = extract_utterance(filename)
            distribution, messages_count = get_distributions(extracted)
            distribution = np.array(distribution)
            for t in range(3):
                temp_key = key + memory_comp + str(t)
                plot_utterance(distribution / messages_count[t], t, output_dir=output_dir, key=temp_key)


In [ ]:
plot_unigram_statistics(filenames, output_dir)
plot_unigram_statistics(filenames_memory_comp, output_dir, True)

In [ ]:
def plot_bigram_statistics(filename, output_dir=False, key=False):
    """
    FOR FIGURE 3b
    
    Bigram counts for prosocial agents communicating via the linguistic channel, sorted by frequency.
    """
    
    extracted = extract_utterance(filename)
    extracted_a = extracted[::2]
    extracted_b = extracted[1::2]
    extracted_a = np.array([list(map(str, msg)) for sublist in extracted_a for msg in sublist])
    extracted_b = np.array([list(map(str, msg)) for sublist in extracted_b for msg in sublist])

    bigrams_a = []
    bigrams_b = []
    for i in range(extracted_a.shape[1] - 1):
        new_bigrams_a = list(np.core.defchararray.add(extracted_a[:, i], extracted_a[:, i + 1]))
        new_bigrams_b = list(np.core.defchararray.add(extracted_b[:, i], extracted_b[:, i + 1]))
        bigrams_a += new_bigrams_a
        bigrams_b += new_bigrams_b

    for i in range(2):
        
        plt.clf()
        plt.ylim([0, 0.3])
        plt.xlim([0, 100])
        bigrams = [bigrams_a, bigrams_b][i]
        
        unique, counts = np.unique(bigrams, return_counts=True)
        counts, unique = order_lists(counts, unique)
        x = np.arange(len(counts))
        # normalization
        counts = counts / sum(counts)
        plt.bar(x, counts)
        if output_dir and key:
            output = '{}bigrams{}_{}.png'.format(output_dir, key, i)
            plt.savefig(output)

    
def plot_bigram_statistics_all(filenames, output_dir=False, memory_comp=False):
    """
    FOR FIGURE 3a
    
    Unigram statistics of symbol usage broken down by turn and by position within the utterance
    for prosocial agents communicating via the linguistic channel.
    """
    
    memory_comp = ['', 'memory_comp'][memory_comp]
    
    for sociality, channel in gen_iter(exclude_channels=['proposal', 'none']):
        key = sociality + '_' + channel
        filename = filenames.get(key, None)
        if filename:
            plot_bigram_statistics(filename, output_dir=output_dir, key=key + memory_comp)


In [ ]:
plot_bigram_statistics_all(filenames, output_dir=output_dir)
plot_bigram_statistics_all(filenames_memory_comp, output_dir=output_dir, memory_comp=True)

In [ ]:
"""
How many messages did agents send?
"""

filename = filenames['prosoc_linguistic']
extracted = extract_utterance(filename)
distribution, messages_count = get_distributions(extracted, n=10)
agent1_count = sum(messages_count[::2])
agent2_count = sum(messages_count[1::2])
print('agent 1 count', agent1_count)
print('agent 2 count', agent2_count)
print('ratio', agent1_count/agent2_count)

In [ ]:
"""
Some additional calculations.
"""

filename = filenames['prosoc_both']
extracted = extract_utterance(filename)
distribution, messages_count = get_distributions(extracted, n=10)
agent1_count = sum(messages_count[::2])
agent2_count = sum(messages_count[1::2])
print('agent 1 count', agent1_count)
print('agent 2 count', agent2_count)
print('ratio', agent1_count/agent2_count)